In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network
model = Network('MyNetwork')

# Block 1: nodes
node1 = Delay(model, 'SlowDelay')
node2 = Queue(model, 'FCFSQueue', SchedStrategy.FCFS)
node3 = Queue(model, 'SIROQueue', SchedStrategy.SIRO)  # Service In Random Order
node4 = Queue(model, 'PSQueue', SchedStrategy.PS)      # Processor Sharing
node5 = Queue(model, 'HOLQueue', SchedStrategy.HOL)    # Head-of-Line priority
node6 = Delay(model, 'FastDelay')

In [3]:
# Block 2: classes with priorities
# Priority: 0=low, 1=high - Class2 has higher priority
jobclass1 = ClosedClass(model, 'Class1', 18, node1, 0)  # Low priority
jobclass2 = ClosedClass(model, 'Class2', 18, node1, 1)  # High priority
jobclass3 = ClosedClass(model, 'Class3', 18, node1, 0)  # Low priority

In [4]:
# Set service times - identical at delay stations for fair comparison
# SlowDelay: all classes same service time
node1.set_service(jobclass1, Exp.fit_mean(10.0))
node1.set_service(jobclass2, Exp.fit_mean(10.0))
node1.set_service(jobclass3, Exp.fit_mean(10.0))

# FCFSQueue: different service times per class
node2.set_service(jobclass1, Exp.fit_mean(0.3))
node2.set_service(jobclass2, Exp.fit_mean(0.5))
node2.set_service(jobclass3, Exp.fit_mean(0.6))

# SIROQueue: different service times per class
node3.set_service(jobclass1, Exp.fit_mean(1.1))
node3.set_service(jobclass2, Exp.fit_mean(1.3))
node3.set_service(jobclass3, Exp.fit_mean(1.5))

# PSQueue: different service times per class
node4.set_service(jobclass1, Exp.fit_mean(1.0))
node4.set_service(jobclass2, Exp.fit_mean(1.1))
node4.set_service(jobclass3, Exp.fit_mean(1.9))

# HOLQueue: different service times per class - priority will matter here
node5.set_service(jobclass1, Exp.fit_mean(2.5))
node5.set_service(jobclass2, Exp.fit_mean(1.9))
node5.set_service(jobclass3, Exp.fit_mean(4.3))

# FastDelay: all classes same service time
node6.set_service(jobclass1, Exp.fit_mean(1.0))
node6.set_service(jobclass2, Exp.fit_mean(1.0))
node6.set_service(jobclass3, Exp.fit_mean(1.0))

In [5]:
# Create cyclic routing: SlowDelay -> FCFS -> SIRO -> PS -> HOL -> FastDelay -> SlowDelay
P = model.init_routing_matrix()

# All classes follow the same cyclic route
for jobclass in [jobclass1, jobclass2, jobclass3]:
    P.set(jobclass, jobclass, node1, node2, 1.0)  # SlowDelay -> FCFSQueue
    P.set(jobclass, jobclass, node2, node3, 1.0)  # FCFSQueue -> SIROQueue
    P.set(jobclass, jobclass, node3, node4, 1.0)  # SIROQueue -> PSQueue
    P.set(jobclass, jobclass, node4, node5, 1.0)  # PSQueue -> HOLQueue
    P.set(jobclass, jobclass, node5, node6, 1.0)  # HOLQueue -> FastDelay
    P.set(jobclass, jobclass, node6, node1, 1.0)  # FastDelay -> SlowDelay

model.link(P)

In [6]:
# MVA with seed=23000 to match JAR test
solver = MVA(model, seed=23000, verbose=VerboseLevel.SILENT)
avg_table = solver.avg_table()

  Station JobClass       QLen       Util       RespT      ResidT       ArvR       Tput
SlowDelay   Class1 3.8265e+00 3.8265e+00     10.0000     10.0000 3.8265e-01 3.8265e-01
SlowDelay   Class2 5.2631e-04 5.2631e-04     10.0000     10.0000 5.2631e-05 5.2631e-05
SlowDelay   Class3 2.2633e+00 2.2633e+00     10.0000     10.0000 2.2633e-01 2.2633e-01
FCFSQueue   Class1 1.7234e-01 1.1480e-01      0.4504      0.4504 3.8265e-01 3.8265e-01
FCFSQueue   Class2 3.4381e-05 2.6316e-05      0.6532      0.6532 5.2631e-05 5.2631e-05
FCFSQueue   Class3 1.6921e-01 1.3580e-01      0.7476      0.7476 2.2633e-01 2.2633e-01
SIROQueue   Class1 1.8055e+00 4.2092e-01      4.7184      4.7184 3.8265e-01 3.8265e-01
SIROQueue   Class2 2.6467e-04 6.8421e-05      5.0287      5.0287 5.2631e-05 5.2631e-05
SIROQueue   Class3 1.1615e+00 3.3950e-01      5.1320      5.1320 2.2633e-01 2.2633e-01
  PSQueue   Class1 1.8918e+00 3.8265e-01      4.9438      4.9438 3.8265e-01 3.8265e-01
  PSQueue   Class2 2.8622e-04 5.7895e-05   

In [7]:
# SSA with seed=23000, samples=10000 to match JAR test
solver = SSA(model, seed=23000, samples=10000, verbose=VerboseLevel.SILENT)
avg_table = solver.avg_table()

  Station JobClass    QLen   Util    RespT   ResidT   ArvR   Tput
SlowDelay   Class1  1.2631 1.2631   9.5694   9.5694 0.1320 0.1320
SlowDelay   Class2  1.6874 1.6874   9.6172   9.6172 0.1755 0.1755
SlowDelay   Class3  0.7332 0.7332   9.7338   9.7338 0.0753 0.0753
FCFSQueue   Class1  0.0500 0.0377   0.3792   0.3792 0.1320 0.1320
FCFSQueue   Class2  0.1257 0.0880   0.7167   0.7167 0.1755 0.1755
FCFSQueue   Class3  0.0650 0.0464   0.8632   0.8632 0.0753 0.0753
SIROQueue   Class1  0.4873 0.1414   3.6917   3.6917 0.1320 0.1320
SIROQueue   Class2  0.6330 0.2192   3.6076   3.6076 0.1755 0.1755
SIROQueue   Class3  0.4202 0.1133   5.5787   5.5787 0.0753 0.0753
  PSQueue   Class1  0.2645 0.1330   2.0039   2.0039 0.1320 0.1320
  PSQueue   Class2  0.4044 0.1943   2.3050   2.3050 0.1755 0.1755
  PSQueue   Class3  0.2808 0.1367   3.7395   3.7395 0.0751 0.0751
 HOLQueue   Class1 15.8111 0.3336 122.7526 122.7526 0.1288 0.1288
 HOLQueue   Class2 14.9791 0.3162  86.8367  86.8367 0.1725 0.1725
 HOLQueue 